# Recurrent Neural Networks(RNN)

##Imports

In [ ]:
import random
import numpy as np

# for network :
import tensorflow as tf 
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation 
# Long Short Term Memory(recurrent layer with memory), memory our models
# Dense for hidden layers
# Activation layer for the output layer

# for compiling
from keras.optimizers import RMSprop

## Dataset

In [ ]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding = 'utf-8').lower() 
# rb = read binary

### Convert this text into a numerical format  
NN cannot work with sentences. We can pass a numpy array into our network and say predict the next character.

In [ ]:
len(text)

1115394

In [ ]:
len(text)-500001

615393

In [ ]:
# Dataset is too big, selecting only a part of this dataset to train our model
# instead of using the whole text
start = random.randint(0, len(text)-500001)
print(start)
text = text[start:start + 500000]
#text = text[300000:start + 800000]

414904


In [ ]:
# set of all the characters in the set
characters = sorted(set(text))
characters

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [ ]:
# c : character - key
# i : index - value
# enumerate : assigns one number to each character in this set
char_to_index = dict((c, i)for i, c in enumerate(characters))
char_to_index

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'a': 13,
 'b': 14,
 'c': 15,
 'd': 16,
 'e': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'k': 23,
 'l': 24,
 'm': 25,
 'n': 26,
 'o': 27,
 'p': 28,
 'q': 29,
 'r': 30,
 's': 31,
 't': 32,
 'u': 33,
 'v': 34,
 'w': 35,
 'x': 36,
 'y': 37,
 'z': 38}

In [ ]:
# the other way around
index_to_char = dict((i, c)for i, c in enumerate(characters))
index_to_char

{0: '\n',
 1: ' ',
 2: '!',
 3: '$',
 4: '&',
 5: "'",
 6: ',',
 7: '-',
 8: '.',
 9: '3',
 10: ':',
 11: ';',
 12: '?',
 13: 'a',
 14: 'b',
 15: 'c',
 16: 'd',
 17: 'e',
 18: 'f',
 19: 'g',
 20: 'h',
 21: 'i',
 22: 'j',
 23: 'k',
 24: 'l',
 25: 'm',
 26: 'n',
 27: 'o',
 28: 'p',
 29: 'q',
 30: 'r',
 31: 's',
 32: 't',
 33: 'u',
 34: 'v',
 35: 'w',
 36: 'x',
 37: 'y',
 38: 'z'}

##Training Data

In [ ]:
# For example, if the sentence is 'how are yo'
# the next character would be a 'u

# how many letters or characters are we 
# gonna use as features or feature data in order to
# predict the next character?

# you can say predict the next character based on
# the last 5 or 500 characters

# Gleichgewicht, Balance
# careful : network shouldnt be relying on too much data
# but you also need to end up with a reasonable text

SEQ_LENGTH = 40 
STEP_SIZE = 3 
# how many characters are we going 
# to shift to the next sentence 
# SEQ_LENGTH = 5
# Text : Hello World
# First Sequence : Hello
# After shifting 3 characters
# Second Sequnce : lo wo

sentences = [] # features
next_characters = [] # targets


In [ ]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
  # i : sequencelerin başlangıç konumları
  sentences.append(text[i:i+SEQ_LENGTH]) 
  next_characters.append(text[i+SEQ_LENGTH]) # always the next correct letter of the sentences
sentences[0:5]

["t's myself.\ngive me the glass, and there",
 ' myself.\ngive me the glass, and therein ',
 'self.\ngive me the glass, and therein wil',
 'f.\ngive me the glass, and therein will i',
 'give me the glass, and therein will i re']

In [ ]:
next_characters[0:5]

['i', 'w', 'l', ' ', 'a']

### Converting the training data into numpy array (numerical format)

In [ ]:
# One dimension for all the possible sentences
# One dimension for all the individual positions in these sentences
# One dimesion for all the possible characters
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype = np.bool)
x

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [ ]:
# target data
# which would be the next character for which sentence
y = np.zeros((len(sentences), len(characters)), dtype = np.bool)
y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
for i, sentence in enumerate(sentences) : # i : index of sentence
  for t, character in enumerate(sentence): # t : index of character
    x[i, t, char_to_index[character]] = 1 # that char occurs at that position in that sentence -->true
  y[i, char_to_index[next_characters[i]]] = 1

In [ ]:
x

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False,  True, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False,  True, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False,  True, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [ ]:
y

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

## Building the Network
Feed the training data into a RNN so it can predict the next character

In [ ]:
model = Sequential()
# first layer : memory, input
# memory of our network : it will remember the past couple of
# characters, the important and relevant ones
# remember the input data, that fed into network few iterations ago
# LSTM(#Neurons, input_shape)
model.add(LSTM(128, input_shape = (SEQ_LENGTH, len(characters))))

In [ ]:
# second layer : dense
model.add(Dense(len(characters)))

In [ ]:
# last layer : activation
model.add(Activation('softmax'))
# softmax : scales the output so that all the values add up to 1
# the output is always a probability of how likely a certain character is
# going to be the next character

# base sequence with multiple characters with a different likelihood
# next character : 70% k, 20% x, ...

In [ ]:
# lr = learning rate
model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(lr = 0.01))

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
# fit it on the training data
# batch_size(power of 2) = how many examples are we going to put into the network at once
# epochs = how mant times our network is going to see the same data over and over again
model.fit(x, y, batch_size =256,epochs=4)

Epoch 1/4
651/651 [==============================] - 115s 171ms/step - loss: 2.0975
Epoch 2/4
651/651 [==============================] - 126s 193ms/step - loss: 1.6800
Epoch 3/4
651/651 [==============================] - 114s 175ms/step - loss: 1.5667
Epoch 4/4
651/651 [==============================] - 113s 174ms/step - loss: 1.5027


In [ ]:
# train once then save them all and later on load them all
# instead training over and over again everytime we run the script, requires a lot of time
model.save('textgenerator.model')

In [ ]:
model = tf.keras.models.load_model('textgenerator.model')

## Get predictions and convert these predictions into text generation

In [ ]:
# takes the predictions of our model
# and picks one specific character 

# this choice will be either conservative
# or experimental
# high tempature --> risky, experimental, more creative
# low tempature --> safe, conservative

def sample(preds, tempature = 1.0) :
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / tempature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [ ]:
def generate_text(length, tempature):
  start_index = random.randint(0, len(text)- SEQ_LENGTH - 1)
  generated = ''
  # basis for our text : first 40 characters from original text
  sentence = text[start_index : start_index + SEQ_LENGTH]
  generated += sentence

  for i in range(length):
    x = np.zeros((1, SEQ_LENGTH, len(characters)))
    for t, character in enumerate(sentence): # index - char
      x[0, t, char_to_index[character]] = 1

    predictions = model.predict(x, verbose = 0)[0]
    next_index = sample(predictions, tempature)
    next_character = index_to_char[next_index]

    generated += next_character
    sentence = sentence[1:] + next_character

  return generated

## Results

In [ ]:
print('--------0.2--------')
print(generate_text(300, 0.2))

print('--------0.4--------')
print(generate_text(300, 0.4))

print('--------0.6--------')
print(generate_text(300, 0.6))

print('--------0.8--------')
print(generate_text(300, 0.8))

print('--------1--------')
print(generate_text(300, 1.0))

--------0.2--------
?

nurse:
marry, that, i think, be young she will be such a courter
to the heart to man the sin to the father
to conder to promined in the prosent to the hour
and the prove to man the pardon to the fire
to the dear to the father to the proolans:
the senter be the pasion and the father,
the thou art to the sun in the pardon to be the sin
t
--------0.4--------
ly she weeps for tybalt's death,
and the more of the son and to the father,
and the alland the mother and mine of my brother.

leontes:
now, i thank you pardon in the sword,
in some be them to see the gally more him.

king edward iv:
now the to the heart and from the state it do the house
to passon to sire that she gone his solder
to many
--------0.6--------
in the current, made it more
violent and bornible him it is dount,
to the field and from the pase frem thy sworns,
the sorrow what i stall stay, when thy son,
to my find with the grave, which is so like.

juliet:
now then me that mount measure a true merry,
